# Learning a Communication Channel

When using non-adaptive FPGA ensembles (i.e., the `learning_rate` is set to 0), the ensemble's decoded output function can be defined by providing the `FpgaPesEnsembleNetwork` constructor with the `function` keyword (e.g., `function=lambda t: sin(t)`). Adaptive (learning) FPGA ensembles augment this feature by allowing the use of error-driving learning to compute these function in an "online" manner.

## Step 1: Set up the Python Imports

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import time

import nengo
from nengo.processes import WhiteSignal
from nengo.solvers import NoSolver

import nengo_fpga
from nengo_fpga.networks import FpgaPesEnsembleNetwork

In [2]:
def make_native_model(n_neurons, dimensions=1, 
                      neuron_type=nengo.neurons.RectifiedLinear(), 
                      seed=1):
    model = nengo.Network(seed=seed)
    
    with model:
        # White-noise reference signal
        input_node = nengo.Node(WhiteSignal(60, high=5),
                                size_out=dimensions)
        output_node = nengo.Node(size_in=dimensions)
            
        ens = nengo.Ensemble(n_neurons, dimensions=dimensions, 
                             neuron_type=neuron_type)
        nengo.Connection(input_node, ens)
        conn = nengo.Connection(
            ens.neurons, output_node, 
            transform=np.random.random((dimensions, n_neurons)))
        conn.learning_rule_type = nengo.PES(learning_rate=5e-5)
        
        error = nengo.Node(size_in=dimensions)
        nengo.Connection(output_node, error)
        nengo.Connection(input_node, error, transform=-1)
        nengo.Connection(error, conn.learning_rule)
    
    return model

In [3]:
def make_fpga_model(n_neurons, dimensions=1, board='de1', 
                    neuron_type=nengo.neurons.RectifiedLinear(), seed=1):
    # Create a nengo network object to which we can add
    # ensembles, connections, etc.
    model = nengo.Network(seed=seed)

    with model:
        # White-noise reference signal
        input_node = nengo.Node(WhiteSignal(60, high=5),
                                size_out=dimensions)

        # Remote FPGA neural ensemble
        fpga_ens = FpgaPesEnsembleNetwork(
            board, # The board to use (from above)
            n_neurons=n_neurons, # The number of neurons to use in the ensemble
            # Number of dimensions for the ensemble to represent
            dimensions=dimensions,
            learning_rate=5e-5
        )

        fpga_ens.ensemble.neuron_type = neuron_type
        fpga_ens.connection.solver = NoSolver(
            np.random.random((n_neurons, dimensions)))

        # Connect the input to the FPGA ensemble
        nengo.Connection(input_node, fpga_ens.input)

        # Create a Nengo node to calculate the error signal
        error = nengo.Node(size_in=dimensions)

        # Compute the error (error = actual - target) using transforms
        # on the connections to the error ensemble
        nengo.Connection(fpga_ens.output, error)
        nengo.Connection(input_node, error, transform=-1)

        # Project error to the adaptive neural ensemble on the FPGA
        nengo.Connection(error, fpga_ens.error)
    return model

In [7]:
# nengo.utils.logging.log('info')

times_fpga = []
times_native = []

dimensions = 1
n_neuron_list = [100, 500, 1000, 5000, 10000, 15000]
# dimensions = 8
# n_neuron_list = [100, 500, 1000, 2000]
# neuron_type = nengo.neurons.SpikingRectifiedLinear()

for n_neurons in n_neuron_list:
    print(">> CREATE FPGA n_neurons=%i" % n_neurons)
    model = make_fpga_model(n_neurons, board='de1',
                            dimensions=dimensions,
                            neuron_type=neuron_type)
    sim_fpga = nengo_fpga.Simulator(model)
    print(">> RUNNING FPGA n_neurons=%i" % n_neurons)
    t_start_fpga = time.time()
    sim_fpga.run(10)
    t_end_fpga = time.time()
    times_fpga.append(t_end_fpga - t_start_fpga)
    
    print(">> CREATE NATIVE n_neurons=%i" % n_neurons)
    model = make_native_model(n_neurons,
                              dimensions=dimensions,
                              neuron_type=neuron_type)
    sim_native = nengo.Simulator(model)
    print(">> RUNNING NATIVE n_neurons=%i" % n_neurons)
    t_start_native = time.time()
    sim_native.run(10)
    t_end_native = time.time()
    times_native.append(t_end_native - t_start_native)
    
    print(">> TAKING 10")
    time.sleep(20)
    print("<< NEXT")

>> CREATE FPGA n_neurons=100
>> RUNNING FPGA n_neurons=100
>> CREATE NATIVE n_neurons=100
>> RUNNING NATIVE n_neurons=100
>> TAKING 10
<< NEXT
>> CREATE FPGA n_neurons=500
>> RUNNING FPGA n_neurons=500
>> CREATE NATIVE n_neurons=500
>> RUNNING NATIVE n_neurons=500
>> TAKING 10
<< NEXT
>> CREATE FPGA n_neurons=1000
>> RUNNING FPGA n_neurons=1000
>> CREATE NATIVE n_neurons=1000
>> RUNNING NATIVE n_neurons=1000
>> TAKING 10
<< NEXT
>> CREATE FPGA n_neurons=5000
>> RUNNING FPGA n_neurons=5000
>> CREATE NATIVE n_neurons=5000
>> RUNNING NATIVE n_neurons=5000
>> TAKING 10
<< NEXT
>> CREATE FPGA n_neurons=10000
>> RUNNING FPGA n_neurons=10000
>> CREATE NATIVE n_neurons=10000
>> RUNNING NATIVE n_neurons=10000
>> TAKING 10
<< NEXT
>> CREATE FPGA n_neurons=15000
>> RUNNING FPGA n_neurons=15000
>> CREATE NATIVE n_neurons=15000
>> RUNNING NATIVE n_neurons=15000
>> TAKING 10
<< NEXT


In [8]:
print(times_fpga)
print(times_native)

[16.053556203842163, 14.857270956039429, 15.28602647781372, 15.035904884338379, 14.617046356201172, 15.124614953994751]
[1.8270559310913086, 1.9985826015472412, 2.0909831523895264, 4.19295334815979, 7.673001289367676, 11.380320072174072]


In [ ]:
# Plot figure
plt.figure(figsize=(16, 8))
plt.plot(sim.trange(), sim.data[input_p], c='k', label='Input')
plt.plot(sim.trange(), sim.data[output_p], c='r', label='FPGA Output')
plt.ylim(-1.5, 1.5)
plt.legend(loc='upper right')
plt.xlabel("Sim time (s)")
plt.title("Random Initialization Without Learning");